In [ ]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb

import concurrent.futures, dateutil.tz, json, os, re, requests, subprocess, sys

if not os.path.exists('python-utils'):
    subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
exec_ipynb('python-utils/esdr-library.ipynb')

# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [ ]:
try:
    import mysql.connector
except:
    subprocess_check('conda install -y mysql-connector-python', verbose=True)
    import mysql.connector


In [ ]:
# First time uploading, create a new client like so:

# Esdr.save_client('esdr-auth-argos-uploader.json', 'Argos uploader')

# and then follow the directions it prints, which include visiting esdr.cmucreatelab.org and creating
# a client with given parameters, and also editing json to include your
# username and password

# Do not add esdr-auth-*.json to the git repo
# !echo 'esdr-auth-*.json' >>.gitignore

In [ ]:
esdr = Esdr('esdr-auth-argos-uploader.json')

product = esdr.get_or_create_product('ArgosSpectrometer',
                                     vendor='Argos',
                                     description='Argos Open Path and Hound Air Spectrometers',
                                     default_channel_specs={
                                         'version':1,
                                         'channels': {
                                             'signal_strength': {
                                                 'prettyName': 'Signal Strength'
                                             },
                                             'qa_complete': {
                                                 'prettyName': 'QA complete'
                                             },
                                             'status': {
                                                 'prettyName': 'Status'
                                             },
                                             'SO2': {
                                                'prettyName': 'SO2 PPB',
                                                'units': 'PPB'
                                             },
                                             'benzene': {
                                                 'prettyName': 'Benzene PPB',
                                                 'units': 'PPB'
                                             },
                                         }
                                     })                           

device = esdr.get_or_create_device(product, 'Glassport')

feed = esdr.get_or_create_feed(device, 40.326009, -79.881742)

In [ ]:
# Do not add argos-auth.json to the git repo
#!echo 'argos-auth.json' >>.gitignore

argos_auth = json.load(open('argos-auth.json'))

integration_time = 5 * 60 # seconds

mysql2esdr_colmap = {
    'signalstrength': 'signal_strength',
    'ben': 'benzene',
    'so2': 'so2',
    'status': 'status'
}


# Note that database timezone is Pacific Local, even though sensor is in Eastern Time Zone
database_timezone = dateutil.tz.gettz('America/Los_Angeles') # Pacific local time (daylight savings observed in summer)

In [ ]:
cnx = mysql.connector.connect(
    host=argos_auth['hostname'],
    user=argos_auth['username'],
    password=argos_auth['password'],
    database='argos'
)

cur = cnx.cursor(buffered=True)

In [ ]:
# Returns end record
def download_and_upload(start_record):
    cur.execute('SELECT * FROM pittsburg ORDER BY sampledate LIMIT 100000 OFFSET %s;', (start_record,))

    assert(cur.column_names[0] == 'sampledate')
    assert(cur.column_names[1] == 'actualdate')

    actual_sql_data_cols = sorted(cur.column_names[2:])
    expected_sql_data_cols = sorted(mysql2esdr_colmap.keys())

    if actual_sql_data_cols == expected_sql_data_cols:
        print('Data columns: %s' % mysql2esdr_colmap)
    else:
        print('WARNING: Expected sql data columns %s but found %s' % (expected_sql_data_cols, actual_sql_data_cols))
    
    data = []

    for sample in cur:
        assert(cur.column_names[0] == 'sampledate')
        sample_end_epochtime = sample[0].replace(tzinfo=database_timezone).timestamp()
        sample_midpoint_epochtime = sample_end_epochtime - 0.5 * integration_time
        sample_data = list(sample[2:])
        data.append([sample_midpoint_epochtime] + sample_data)
        
    channel_names = [mysql2esdr_colmap[c] for c in cur.column_names[2:]]
    esdr.upload(feed, {'channel_names': channel_names, 'data':data})

    print('Starting at record %d, captured and uploaded %d samples of %d channels each, with last sample %d seconds ago' % (start_record, len(data), len(data[0])-1, time.time() - data[-1][0]))
    return start_record + len(data)



In [ ]:
end_record = 0

while True:
    end_record = download_and_upload(max(0, end_record - 500))
    time.sleep(60 - (time.time() % 60))
